In [1]:
push!(LOAD_PATH,".")

4-element Vector{String}:
 "@"
 "@v#.#"
 "@stdlib"
 "."

In [2]:
using Typeside, Chakra, PitchADT, MidiPitch, ChakraImp, Viewpoints, Models

┌ Info: Precompiling Models [top-level]
└ @ Base loading.jl:1342


In [3]:
o1 = set(delim(),:pitch,NoteNumber(60))
o2 = set(delim(),:pitch,NoteNumber(62))
o3 = set(delim(),:pitch,NoteNumber(68))
o4 = set(delim(),:pitch,NoteNumber(60))
o5 = set(delim(),:pitch,NoteNumber(62))
o6 = set(delim(),:pitch,NoteNumber(70))

ChakraImp.Obj(ChakraImp.Id[], Dict{Symbol, Any}(:pitch => NoteNumber(70)))

In [4]:
s = [o1,o2,o3,o4,o5,o6]

6-element Vector{ChakraImp.Obj}:
 ChakraImp.Obj(ChakraImp.Id[], Dict{Symbol, Any}(:pitch => NoteNumber(60)))
 ChakraImp.Obj(ChakraImp.Id[], Dict{Symbol, Any}(:pitch => NoteNumber(62)))
 ChakraImp.Obj(ChakraImp.Id[], Dict{Symbol, Any}(:pitch => NoteNumber(68)))
 ChakraImp.Obj(ChakraImp.Id[], Dict{Symbol, Any}(:pitch => NoteNumber(60)))
 ChakraImp.Obj(ChakraImp.Id[], Dict{Symbol, Any}(:pitch => NoteNumber(62)))
 ChakraImp.Obj(ChakraImp.Id[], Dict{Symbol, Any}(:pitch => NoteNumber(70)))

In [5]:
v = AtomicViewpoint(Pitch,:pitch)
vp_type(v)
vp_apply(v,s)

NoteNumber(60)

In [6]:
v2 = LinkedViewpoint(v,DelayedViewpoint(v,1))
#vp_type(v2)
vp_apply(v2,s)

(NoteNumber(60), NoteNumber(62))

In [7]:
v3 = DerivedViewpoint(PitchInterval,v2,PitchADT.diff)
vp_type(v3)
vp_apply(v3,s)

NoteInterval(2)

In [8]:
v4 = DerivedViewpoint(Pitch,v3,x->PitchADT.shift(x,NoteNumber(40)))
vp_type(v4)
vp_apply(v4,s)

NoteNumber(42)

In [9]:
m = Models.NGramModel{NoteNumber,3}(vp_map(v,s))

Models.NGramModel{NoteNumber, 3}(Dict{Vector{NoteNumber}, Int64}([NoteNumber(68), NoteNumber(60), NoteNumber(62)] => 1, [NoteNumber(62), NoteNumber(68), NoteNumber(60)] => 1, [NoteNumber(60), NoteNumber(62), NoteNumber(70)] => 1, [NoteNumber(60), NoteNumber(62), NoteNumber(68)] => 1), Set(NoteNumber[NoteNumber(60), NoteNumber(62), NoteNumber(70), NoteNumber(68)]))

In [10]:
Models.ml(Models.NGram{NoteNumber,3}(NoteNumber(68),[NoteNumber(60),NoteNumber(62)]),m)

1.0

In [11]:
m.elems

Set{NoteNumber} with 4 elements:
  NoteNumber(60)
  NoteNumber(62)
  NoteNumber(70)
  NoteNumber(68)

In [12]:
m.db

Dict{Vector{NoteNumber}, Int64} with 4 entries:
  [NoteNumber(68), NoteNumber(60), NoteNumber(62)] => 1
  [NoteNumber(62), NoteNumber(68), NoteNumber(60)] => 1
  [NoteNumber(60), NoteNumber(62), NoteNumber(70)] => 1
  [NoteNumber(60), NoteNumber(62), NoteNumber(68)] => 1

In [13]:
struct pc <: Pitch
    val::Char
end
Typeside.@associatedType(:pc,pc)
pcv = AtomicViewpoint(pc,:pc)
e1 = set(delim(),:pc,pc('G'))
e2 = set(delim(),:pc,pc('G'))
e3 = set(delim(),:pc,pc('D'))
e4 = set(delim(),:pc,pc('B'))
e5 = set(delim(),:pc,pc('A'))
e6 = set(delim(),:pc,pc('G'))
e7 = set(delim(),:pc,pc('G'))
e8 = set(delim(),:pc,pc('A'))
e9 = set(delim(),:pc,pc('B'))
e10 = set(delim(),:pc,pc('A'))
choral = [e10,e9,e8,e7,e6,e5,e4,e3,e2,e1]
choral_model = Models.HGramModel{pc,3}(vp_map(pcv,choral))

Models.HGramModel{pc, 3}(Dict{Vector{pc}, Int64}([pc('G'), pc('A')] => 1, [pc('A'), pc('B'), pc('A')] => 1, [pc('G')] => 4, [pc('A'), pc('G')] => 1, [pc('G'), pc('G'), pc('A')] => 1, [pc('A'), pc('B')] => 2, [pc('A'), pc('B'), pc('D')] => 1, [pc('D'), pc('G'), pc('G')] => 1, [pc('G'), pc('A'), pc('B')] => 1, [pc('B'), pc('D')] => 1…), Set(pc[pc('D'), pc('B'), pc('G'), pc('A')]))

In [14]:
Models.ml(Models.NGram{pc,1}(pc('E'),pc[]),choral_model)

0.0

In [15]:
Models.ppm(Models.NGram{pc,2}(pc('A'),[pc('E')]),choral_model)

0.25